In [1]:
#Importing dependencies/libraries

from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np

In [2]:
#Changing approach. Instead of going through several city/locations in a for loop, we will scrape per city/location.
#Scraping per city/location saves allows us to use the data as soon as its scraped.

# #ON
# location_on = ["greater-sudbury", "hamilton", "kingston", "london", "niagara-falls",
#             "ottawa", "toronto", "windsor", "kitchener", "peterborough", 
#             "sault-ste-marie", "thunder-bay"]

# #AB
# location_ab = ["edmonton", "calgary", "lethbridge", "red-deer"]

# #NS
# location_ns = ["halifax"]

# #QC
# location_qc = ["montreal", "sherbrooke", "trois-rivieres", "quebec"]

# #BC
# location_bc = ["vancouver", "victoria"]

# #MB
# location_mb = ["winnipeg"]

#------ADJUST LOCATION VARIABLE------
location_qc = ['montreal']
prov = 'qc'

#Yelp shows 10 restaurants (not including sponsored ones) per page. 
#The web page can be changed by manipulating the number at after the'start=' line of the url.
pages = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230]

In [3]:
#Scrape links for each restaurant from the yelp's front page

#Empty lists to hold the values
restaurant_links = []

#Webscraping for restaurant links
browser2 = Browser('chrome')

#This for loop changes the page on the url
#------ADJUST LOCATION VARIABLE------
for loc in location_qc:
    
    #This for loop changes the location of the url
    for page in pages:  
        
        #Adjust variable names in URL
        url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc={loc}%2C+{prov}&start={page}'
        browser2.visit(url)
        html = browser2.html
        soup_obj = soup(html, 'html.parser')
        
        res_info = []
        res_info = soup_obj.find_all("div", class_ = "businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY")
        time.sleep(3)
        
        for info in res_info:
            
            try:
                target = info.a['href']
                restaurant_links.append(target)

            except:
                restaurant_links.append("No data or change code")    
                
browser.quit()

KeyboardInterrupt: 

In [6]:
browser2.quit()

In [ ]:
#Visually checking scraping results
restaurant_links

In [ ]:
#Visually checking scraping results
len(restaurant_links)

In [ ]:
#Extracting address from each link, using the link in the list restaurant_links

#Setting up a counter to keep track of progress; this code takes about ~30 mins to run
counter = 1

#Setting up empty lists
names = []
# total_reviews = []
# all_information = []
restaurant_address = []

#Extracting restaurant name and address from restaurant's business page
browser2 = Browser('chrome')

for link in restaurant_links:
    
    print(f'Link # {counter}')
    
    #Visiting the resturant's Yelp page 
    base_url = "https://www.yelp.com"
    res_url = f"https://www.yelp.com{link}"

    browser2.visit(res_url)

    html = browser2.html

    soup_obj = soup(html, 'html.parser')
    time.sleep(3)
    
    #Extracting name
    name = soup_obj.find('h1', class_ = "css-1se8maq")
    if name:
        data = name.text
        names.append(data)

    else:
        names.append("No data or change code")


#     reviews = soup_obj.find('a', class_ = "css-19v1rkv")
#     if reviews:
#         data2 = reviews.text
#         total_reviews.append(data2)
#         print(data2)
#         print("------")
#     else:
#         print("No data or change code")

#     information = soup_obj.find_all('span', class_ = "css-1fdy0l5")
#     if information:
#         for info in information:
#             all_information.append(info.text)
#             print(info.text)

#     else:
#         print("No data or change code")
    
    #Extracting address
    address = soup_obj.find('p', class_ = "css-qyp8bo")

    if address:
        data3 = address.text
        restaurant_address.append(data3)

    else:
        restaurant_address.append("No data or change code")
        
    
    counter += 1

browser.quit()

In [ ]:
#Visually checking scraping results
restaurant_address

In [ ]:
#Visually checking scraping results
len(restaurant_address)

In [ ]:
#Converting and joining restaurant name and address scraped from business page
test_df = pd.DataFrame({
    "restaurant_name" : names,
    "address" : restaurant_address
})
test_df

In [ ]:
#Scraping information from Yelp's front page

#Empty lists to hold the values
names_fps = []
price_range = []
ratings = []
total_reviews = []
category = []
location = []
province = []

#Web scraping
browser2 = Browser('chrome')

#This for loop changes the page on the url
#------ADJUST LOCATION VARIABLE------
for loc in location_qc:
    
    #This for loop changes the location of the url
    for page in pages:  
        
        url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc={loc}%2C+{prov}&start={page}'
        browser2.visit(url)
        html = browser2.html
        soup_obj = soup(html, 'html.parser')      
        
        #Scraping the div that contains the target elements
        res_info = []    
        res_info = soup_obj.find_all("div", class_ = "container__09f24__mpR8_ hoverable__09f24__wQ_on border-color--default__09f24__NPAKY")
        time.sleep(3)
        
        #Extracting the target elements
        for info in res_info:
            #Extracting restaurant name
            try:
                name = info.find("a", class_ = "css-19v1rkv").text
                names_fps.append(name)
            except:
                names_fps.append("No data or change the code")
              
            #Extracting price range
            if info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}).text
                price_range.append(prc_range)
            elif info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}).text
                price_range.append(prc_range)
            else:
                price_range.append("No data or change the code")
            
            #Extracting ratings
            try:
                rati = info.find("span", {"class":"css-gutk1c"}).text
                ratings.append(rati)
            except:
                ratings.append("No data or change the code")
            
            #Extracting total number of reviews
            #Yelp seems to have a different class for reviews in Montreal's restaurant front page, so code is adjusted
#             try: #---------------------> OG code
#                 revs = info.find("span", {"class":"css-chan6m"}).text
#                 total_reviews.append(revs)
#             except:
#                 total_reviews.append("No data or change the code")
            if info.find("span", {"class":"css-chan6m"}):
                revs = info.find("span", {"class":"css-chan6m"}).text
                total_reviews.append(revs)
            elif info.find("span", {"class":"css-8xcil9"}):
                revs = info.find("span", {"class":"css-8xcil9"}).text
                total_reviews.append(revs)
            else:
                total_reviews.append("No data or change the code")
            
            
            #Extracting category
            try:
                categ = info.find("span", {"class":"css-11bijt4"}).text
                category.append(categ)
            except:
                category.append("No data or change the code")
            
            #Assigning location and province.
            #------ADJUST PROVINCE TO APPEND------
            location.append(loc)
            province.append("QC")

browser.quit()

In [ ]:
#Visually checking scraping results
total_reviews

#This list contains valeus that are not representative of 'total reviews'
#Double check after merging and cleaning

In [ ]:
#Visually checking scraping results
len(category)

In [ ]:
#Converting and joining data scraped from front page
test_df2 = pd.DataFrame({
    "restaurant_name" : names_fps,
    "price_range" : price_range,
    "rating" : ratings,
    "total_reviews" : total_reviews,
    "category" : category,
    "location" : location,
    "province" : province
})

test_df2

In [ ]:
#Merging business page and front page dataframe
merged_df = pd.merge(test_df2, test_df, on="restaurant_name", how = "left")
merged_df

In [ ]:
#Cleaning merged dataframe
merged_df.drop_duplicates()
merged_df.replace('No data or change the code', np.nan, inplace = True)
merged_df.dropna(inplace = True)
merged_df = merged_df.reset_index(drop = True)
merged_df['total_reviews'] = merged_df['total_reviews'].str.replace(r'[\(\) reviews]', '', regex=True)

merged_df

In [ ]:
#Double checking
merged_df['total_reviews'].unique()

#The questionable values seems to have been removed after data cleaning

In [ ]:
#Visually checking dataframe
data_checking = merged_df['restaurant_name'].value_counts()
data_checking.head(10)

In [ ]:
#Visually checking dataframe
#------ADJUST VALUE TO BE CHECKED------
merged_df[merged_df.eq('KINTON RAMEN').any(axis=1)]


In [ ]:
#Further cleaning dataframe, each restaurant to be dropped were visually inspected through their Yelp business page
merged_df.drop([24, 97, 69, 107, 91, 51], inplace = True)

In [ ]:
#Visually checking dataframe
data_checking2 = merged_df['restaurant_name'].value_counts()
data_checking2

In [ ]:
#Making an output csv
#------ADJUST CSV NAME------
merged_df.to_csv("yelp_montreal_qc_complete.csv", index = False)